In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas
import datetime
from imdb import IMDb

In [ ]:
movies = pandas.read_csv('movies.csv')
votes = pandas.read_csv('responses.csv')

In [ ]:
# create an instance of the IMDb class
ia = IMDb()

In [ ]:
print(sorted_votes[0][0])

In [ ]:
morvie = sorted_votes[0][0]
i = 0
imdb_movie = ia.search_movie(morvie)[i]
if 'movie' not in imdb_movie['kind']:
    imdb_movie = ia.search_movie(morvie)[i+1]

In [ ]:
print([blorp['kind'] for blorp in ia.search_movie(morvie)])

In [ ]:
imdb_movie.keys()

In [ ]:
imdb_movie.current_info

In [ ]:
imdb_movie['year']